In [17]:
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
import calendar
from pandas import DataFrame

In [30]:
# work on features
cols_to_use =['Period','W', 'ID', 'morning', 'noon', 'afternoon','night']
moovit_data = gpd.read_file(r'output/moovit/APR21WeekDay.shp')[cols_to_use]

moovit_data

,Period,W,ID,morning,noon,afternoon,night
0,APR21,WeekDay,0,1,0,2,0
1,APR21,WeekDay,1,4,4,4,3
2,APR21,WeekDay,2,4,4,4,3
3,APR21,WeekDay,3,1,9,0,2
4,APR21,WeekDay,4,0,0,9,1
...,...,...,...,...,...,...,...
8536,APR21,WeekDay,8760,3,2,1,0
8537,APR21,WeekDay,8761,0,0,0,0
8538,APR21,WeekDay,8762,4,4,4,3
8539,APR21,WeekDay,8763,2,3,2,3


In [32]:
# flat the data so each ped level would be in a new row
def rearrange_data(row):
    r"""
    :param row:
    :return:
    """
    id= row['ID']
    def add_row(day_part,ped_level):
        r"""
        if the ped level is not 9 add new line to the list with ID, day part and ped level
        :param day_part:
        :param ped_level:
        :return:
        """
        if ped_level!=9:
            data_for_ml.append([id,day_part,ped_level])
    [add_row(x,row[x+3]) for x in range(4)]

data_for_ml = []
moovit_data.apply(rearrange_data,axis=1)
ml_data = DataFrame(data_for_ml,columns=['ID','day part','ped_level'])
ml_data

,ID,day part,ped_level
0,0,0,1
1,0,1,0
2,0,2,2
3,0,3,0
4,1,0,4
...,...,...,...
31802,8763,3,3
31803,8764,0,3
31804,8764,1,4
31805,8764,2,2


In [33]:
# Calculate season predictor
month_name = [x for x in calendar.month_abbr]
month = month_name.index(moovit_data['Period'][0][:3].lower().capitalize())
if month in [12, 1, 2]:
    season = 1
elif month in [3, 4, 5]:
    season = 2
elif month in [6, 7, 8]:
    season = 3
else:
    season = 0
ml_data['season'] = month
# Encode W columns
ml_data['day'] = 0 if moovit_data['W'][0]=='WeekDay'else 1
ml_data

,ID,day part,ped_level,season,day
0,0,0,1,4,0
1,0,1,0,4,0
2,0,2,2,4,0
3,0,3,0,4,0
4,1,0,4,4,0
...,...,...,...,...,...
31802,8763,3,3,4,0
31803,8764,0,3,4,0
31804,8764,1,4,4,0
31805,8764,2,2,4,0


In [34]:
network_data = gpd.read_file(r'output/streets_elements/streets_elements.shp')
network_data[]

,oidrechov,length,buildings,businesses,educationa,Health_ser,Leisure_am,Playground,Sport_faci,synagogues,bus_statio,lighting,trees,bike_trail,parks,density,geometry
0,1.0,32.807598,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.705 668574.483, 184351.736 ..."
1,3.0,63.288914,20.415000,0.000000,5,4,1,3,2,3,0.000000,0.015801,0.015801,0,2,18.1,"LINESTRING (179282.603 666005.717, 179343.283 ..."
2,4.0,25.981647,19.000000,0.000000,10,0,0,3,1,1,0.038489,0.038489,0.000000,0,4,15.5,"LINESTRING (180952.197 664967.670, 180964.041 ..."
3,5.0,80.268853,10.202381,0.000000,4,3,0,2,2,3,0.000000,0.037374,0.037374,0,12,10.8,"LINESTRING (180668.486 662875.092, 180748.697 ..."
4,6.0,80.152149,10.500000,0.012476,0,0,0,0,2,0,0.000000,0.024953,0.000000,1,8,25.7,"LINESTRING (182532.174 661455.967, 182514.127 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,9650.0,92.387138,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.064944,0.000000,1,4,16.3,"LINESTRING (180252.127 670097.359, 180342.969 ..."
8747,9651.0,55.993258,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.107156,0.000000,1,2,1.4,"LINESTRING (180204.072 670123.951, 180220.846 ..."
8748,9658.0,1416.463909,12.661667,0.000000,0,0,0,0,0,0,0.002118,0.039535,0.390409,0,23,1.4,"LINESTRING (180765.260 670000.467, 180830.127 ..."
8749,9659.0,720.172594,17.560000,0.000000,0,0,0,0,0,0,0.004166,0.023605,0.279100,0,5,1.4,"LINESTRING (181058.769 671381.112, 181091.908 ..."


In [36]:
result = ml_data.set_index('ID').join(network_data, how='inner')
ml_result = result.drop(columns=['oidrechov','length','geometry'])
ml_result

,day part,ped_level,season,day,buildings,businesses,educationa,Health_ser,Leisure_am,Playground,Sport_faci,synagogues,bus_statio,lighting,trees,bike_trail,parks,density
0,0,1,4,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3
0,1,0,4,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3
0,2,2,4,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3
0,3,0,4,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3
1,0,4,4,0,20.415000,0.000000,5,4,1,3,2,3,0.000000,0.015801,0.015801,0,2,18.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8749,3,4,4,0,17.560000,0.000000,0,0,0,0,0,0,0.004166,0.023605,0.279100,0,5,1.4
8750,0,4,4,0,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4
8750,1,4,4,0,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4
8750,2,4,4,0,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4


In [16]:
gpd.read_file(r'output/moovit/APR21WeekDay.shp')

,Period,W,ID,trechov,shemangli,mslamas,tsug,kkivun,UniqueId,BUFF_DIST,...,noon,afternoon,night,Sp_1_Morn,Sp_2_Noon,Sp_3_Aftrn,Sp_4_Nght,Daily_Avrg,WKT,geometry
0,APR21,WeekDay,0,????? ?????,HARUGEY MALKHOT,336,????,0,507-10001,8,...,0,2,0,2.5,2.9,3.9,2.7,1,MULTILINESTRING ((34.832258364625 32.110044199...,"LINESTRING (184322.705 668574.483, 184351.736 ..."
1,APR21,WeekDay,1,???????,AMSTERDAM,516,????,1,507-10003,8,...,4,4,3,3.1,3.3,3.4,4.3,4,MULTILINESTRING ((34.7789654994423 32.08671155...,"LINESTRING (179282.603 666005.717, 179343.283 ..."
2,APR21,WeekDay,2,???? ????,YIG'AL ALLON,2524,????,0,507-10004,8,...,4,4,3,4.1,3.5,4.3,3.7,4,MULTILINESTRING ((34.7966920352919 32.07740885...,"LINESTRING (180952.197 664967.670, 180964.041 ..."
3,APR21,WeekDay,3,???????,MARGOLIN,2649,????,1,507-10005,8,...,9,0,2,5.1,NaN,4.6,2.8,1,MULTILINESTRING ((34.7937715205782 32.05852872...,"LINESTRING (180668.486 662875.092, 180748.697 ..."
4,APR21,WeekDay,4,???????,HATASSIM DERAKH,2601,???,0,507-10006,8,...,0,9,1,4.2,3.9,NaN,2.9,0,MULTILINESTRING ((34.8135608057952 32.04579371...,"LINESTRING (182532.174 661455.967, 182514.127 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8536,APR21,WeekDay,8760,?????,PROPES,185,????,0,507-21352,8,...,2,1,0,4.0,3.9,5.7,5.6,1,MULTILINESTRING ((34.7890680344983 32.12364294...,"LINESTRING (180252.127 670097.359, 180342.969 ..."
8537,APR21,WeekDay,8761,?????,PROPES,185,????,0,507-21351,8,...,0,0,0,4.9,3.7,5.0,4.0,0,MULTILINESTRING ((34.7885577252564 32.12388106...,"LINESTRING (180204.072 670123.951, 180220.846 ..."
8538,APR21,WeekDay,8762,???? ?????,NAMIR,1011,???,0,507-21364,8,...,4,4,3,3.8,3.6,4.1,4.2,4,MULTILINESTRING ((34.7945094284509 32.12278692...,"LINESTRING (180765.260 670000.467, 180830.127 ..."
8539,APR21,WeekDay,8763,???? ?????,NAMIR,1011,???,0,507-21363,8,...,3,2,3,4.0,4.5,3.9,4.2,2,MULTILINESTRING ((34.797564347166 32.135247199...,"LINESTRING (181058.769 671381.112, 181091.908 ..."
